In [1]:
using LinearAlgebra
using Statistics
using LsqFit
using CSV
using DataFrames
using Base.Iterators
using DataStructures
using JuMP
using Ipopt
using ForwardDiff
using OrderedCollections

#using Gurobi
#using KNITRO

In [2]:
############################
# Parametric Functions
############################

function modelo(t::Vector{T}, kappa::T, theta::T, r0::T) where {T<:Real}
    return @. theta + (r0 - theta)*exp(-kappa*t)
end

modelo (generic function with 1 method)

In [3]:
function val_dist_norm(k::T, theta::T, r::Vector{T}, dt::T) where {T<:Real}
    ts = T.(collect(dt:(dt + length(r) - 1)))
    r_bar = @. theta + exp(-k*ts)*(r[1]-theta)
    y = @. exp(k*ts)*(r - r_bar)

    num = @. exp(-k*ts[2:end-1])*(y[3:end]-y[2:end-1])
    den = @. sqrt(r[2:end-1]*dt)

    dist = num ./ den
    return dist
end


val_dist_norm (generic function with 1 method)

In [4]:
function ols_model(initial_guess::Vector{Float64}, taxas::Vector{Float64}; dt=1.0, 
                   lower_bounds=[0.0;0.0], upper_bounds=[Inf; Inf])
    r0 = taxas[1]
    t_data = collect(0:length(taxas)-1)

    model_fun(t, params) = @. params[2] + (r0 - params[2])*exp(-params[1]*t)
    fit = curve_fit(model_fun, t_data, taxas, initial_guess, lower=lower_bounds, upper=upper_bounds)
    k, theta = fit.param

    dist = val_dist_norm(k, theta, taxas, dt)
    sigma = sqrt(sum((dist .- mean(dist)).^2)/(length(taxas)-1))

    return k, theta, sigma
end

ols_model (generic function with 1 method)

In [5]:
df = CSV.read("selic_over_calib.csv", DataFrame)
taxas_calib = collect(skipmissing(df.CDI))

df, taxas_calib

(22×2 DataFrame
 Row │ Date        CDI     
     │ Date        Float64 
─────┼─────────────────────
   1 │ 2011-12-01   0.1087
   2 │ 2011-12-02   0.1088
   3 │ 2011-12-05   0.1089
   4 │ 2011-12-06   0.1088
   5 │ 2011-12-07   0.1087
   6 │ 2011-12-08   0.1085
   7 │ 2011-12-09   0.1083
   8 │ 2011-12-12   0.1082
   9 │ 2011-12-13   0.1083
  10 │ 2011-12-14   0.1086
  11 │ 2011-12-15   0.1087
  12 │ 2011-12-16   0.1088
  13 │ 2011-12-19   0.1089
  14 │ 2011-12-20   0.109
  15 │ 2011-12-21   0.1089
  16 │ 2011-12-22   0.1088
  17 │ 2011-12-23   0.1088
  18 │ 2011-12-26   0.1085
  19 │ 2011-12-27   0.1086
  20 │ 2011-12-28   0.1085
  21 │ 2011-12-29   0.1087
  22 │ 2011-12-30   0.1087, [0.10869999999999999, 0.10880000000000001, 0.10890000000000001, 0.10880000000000001, 0.10869999999999999, 0.1085, 0.10830000000000001, 0.1082, 0.10830000000000001, 0.10859999999999999  …  0.10890000000000001, 0.109, 0.10890000000000001, 0.10880000000000001, 0.10880000000000001, 0.1085, 0.10859999999999999

In [6]:
val_dist_norm(0.1, 0.12, taxas_calib, 1.0)

20-element Vector{Float64}:
 -0.0032360230599227722
 -0.003872467522567463
 -0.0039061313583914612
 -0.004275035207354183
 -0.0043428308531708055
 -0.004074928577312253
 -0.0034368203955964884
 -0.0027316217835196012
 -0.003302829380931116
 -0.0032694105097083024
 -0.0032360230599227813
 -0.0032026669661579816
 -0.0038388354011439775
 -0.003872467522567465
 -0.0035710772091571007
 -0.004576239656860064
 -0.003336279739209112
 -0.00397355443820347
 -0.0030007627012218203
 -0.0036046187422570184

In [7]:
k, theta, sigma = ols_model([1.0,1.0], taxas_calib)

println("k: $k")
println("theta: $theta")
println("sigma: $sigma")

k: 0.4627449272360901
theta: 0.10865651582838269
sigma: 0.0006599155487009002


In [8]:
#Parametros
a = k*theta
b = k
gamma = 1/2 * sqrt(b^(2) + 2*sigma^(2))

println("a: $a")
println("b: $b")
println("gamma: $gamma")

a: 0.050280251510732024
b: 0.4627449272360901
gamma: 0.23137293416678903


In [9]:
############################
# Parametric Hull-White Functions
############################

function A(t::T, Tt::T, a::T, b::T, gamma::T, sigma::T) where {T<:Real}
    return (-2*a)/(sigma^2) * log((gamma*exp(0.5*b*(Tt-t))) / (gamma*cosh(gamma*(Tt-t)) + 0.5*b*sinh(gamma*(Tt-t))))
end

function C(t::T, Tt::T, b::T, gamma::T, sigma::T) where {T<:Real}
    return (sinh(gamma*(Tt-t))) / (gamma*cosh(gamma*(Tt-t)) + 0.5*b*sinh(gamma*(Tt-t)))
end

function B(t::T, Tt::T, M_total::T, prob::T, r_t::T, a::T, b::T, gamma::T, sigma::T) where {T<:Real}
    return prob * exp(-M_total - A(t, Tt, a, b, gamma, sigma) - C(t, Tt, b, gamma, sigma)*r_t)
end

function PU(r_t::T, Tt::T) where {T<:Real}
    return one(T)/((one(T)+r_t)^(Tt/252))
end

PU (generic function with 1 method)

In [10]:
function cria_vetor_mudancas(min_value::T, max_value::T) where {T<:Real}
    step = T(0.0025)
    num_steps = Int(round((max_value - min_value)/step)) + 1
    return [min_value + (i-1)*step for i in 1:num_steps]
end

cria_vetor_mudancas (generic function with 1 method)

In [11]:
cria_vetor_mudancas(-0.0075, 0.0075)

7-element Vector{Float64}:
 -0.0075
 -0.004999999999999999
 -0.0024999999999999996
  0.0
  0.0025000000000000005
  0.005000000000000001
  0.0075

In [12]:
function gera_mudancas(lista::Vector{T}) where {T<:Real}
    num_negativ = count(x -> x < zero(T), lista)
    num_posit = count(x -> x > zero(T), lista)

    changes = String[]
    for i in num_negativ:-1:1
        push!(changes, "D$(i)")
    end

    # Só adiciona "N" se houver zero no vetor
    if any(x -> x == zero(T), lista)
        push!(changes, "N")
    end

    for i in 1:num_posit
        push!(changes, "U$(i)")
    end

    return changes
end

gera_mudancas (generic function with 1 method)

In [13]:
gera_mudancas(cria_vetor_mudancas(-0.0075, 0.0075))

7-element Vector{String}:
 "D3"
 "D2"
 "D1"
 "N"
 "U1"
 "U2"
 "U3"

In [14]:
function cria_dicionario(min_value::T, max_value::T) where {T<:Real}
    vetor_mudancas = cria_vetor_mudancas(min_value, max_value)
    lista_mudancas = gera_mudancas(vetor_mudancas)
    dicionario = OrderedDict{T,String}()
    for (valor, mudanca) in zip(vetor_mudancas, lista_mudancas)
        dicionario[valor] = mudanca
    end
    return dicionario
end

cria_dicionario (generic function with 1 method)

In [15]:
cria_dicionario(-0.0075, 0.0075)

OrderedDict{Float64, String} with 7 entries:
  -0.0075 => "D3"
  -0.005  => "D2"
  -0.0025 => "D1"
  0.0     => "N"
  0.0025  => "U1"
  0.005   => "U2"
  0.0075  => "U3"

In [16]:
function calcular_cenarios_distintos(min_mudanca::T, max_mudanca::T, n_reunioes::Int) where {T<:Real}
    nomes_mudancas = cria_dicionario(min_mudanca, max_mudanca)

    todas_mudancas = cria_vetor_mudancas(min_mudanca, max_mudanca)
    combinacoes = collect(product((todas_mudancas for i in 1:n_reunioes)...))

    temp_cenarios = Dict{T, Vector{Vector{String}}}()

    for combinacao in combinacoes
        combo = collect(combinacao)
        # Arredonda a soma para 5 casas decimais, por exemplo
        mudanca_acumulada = round(sum(combo), digits=5)
        c_nomes = [nomes_mudancas[x] for x in combo]

        if !haskey(temp_cenarios, mudanca_acumulada)
            temp_cenarios[mudanca_acumulada] = Vector{Vector{String}}()
        end
        push!(temp_cenarios[mudanca_acumulada], c_nomes)
    end

    sorted_keys = sort(collect(keys(temp_cenarios)))
    cenarios = OrderedDict{T,Vector{Vector{String}}}()
    for k in sorted_keys
        cenarios[k] = temp_cenarios[k]
    end

    return cenarios
end

calcular_cenarios_distintos (generic function with 1 method)

In [93]:
calcular_cenarios_distintos(-0.0025, 0.0025, 2)

OrderedDict{Float64, Vector{Vector{String}}} with 5 entries:
  -0.005  => [["D1", "D1"]]
  -0.0025 => [["N", "D1"], ["D1", "N"]]
  0.0     => [["U1", "D1"], ["N", "N"], ["D1", "U1"]]
  0.0025  => [["U1", "N"], ["N", "U1"]]
  0.005   => [["U1", "U1"]]

In [18]:
function caminho_para_probabilidade(cenario::Vector{String}, M::Matrix{T}, mudancas_descricao::Vector{String}, Q_init::Vector{T}) where {T<:Real}
    # Esta função calcula a probabilidade de um cenário específico passo a passo.
    # cenario: ex. ["N", "U1", "D1"] para 3 reuniões
    # M: matriz de transição 3x3
    # mudancas_descricao: ex. ["D1", "N", "U1"] nesta ordem
    # Q_init: vetor distribuição inicial (row vector), ex: [0,1,0] se começamos em N.
    #
    # Supomos que M_ij = P(estado i -> estado j)
    # e Q_init * M dá a próxima distribuição.
    #
    # Para cada passo:
    # 1) Multiplica Q_init * M -> próxima distribuição
    # 2) Identifica o estado desejado neste passo do cenário
    # 3) Extrai a probabilidade correspondente e multiplica na prob total
    # 4) Atualiza Q_init para um vetor que tenha 1 no estado escolhido.

    prob_total = one(T)
    Q_current = copy(Q_init) # distribuição corrente
    
    for estado_desejado in cenario
        #println("Q atual: $Q_current")
        # Próxima distribuição
        next_dist =  M*Q_current
        # Índice do estado desejado
        idx = findfirst(==(estado_desejado), mudancas_descricao)
        if isnothing(idx)
            error("Estado $(estado_desejado) não encontrado em mudancas_descricao.")
        end

        # Extrai a probabilidade do estado desejado
        p = next_dist[idx]
        prob_total *= p

        # Atualiza Q_current para refletir que agora estamos naquele estado
        Q_new = zeros(T, length(mudancas_descricao))
        Q_new[idx] = one(T) # 100% naquele estado
        Q_current = Q_new
    end

    return prob_total
end

caminho_para_probabilidade (generic function with 1 method)

In [19]:
bias_test = [-0.0075, -0.0025]
Q_prev_test = [0.0, 1.0, 0.0]
A_prev_test = [0.5, 0.1, 0.4, 0.9, 0.1, 0.0, 0.25, 0.30, 0.45]

9-element Vector{Float64}:
 0.5
 0.1
 0.4
 0.9
 0.1
 0.0
 0.25
 0.3
 0.45

In [20]:
M = reshape(A_prev_test, 3, 3)

3×3 Matrix{Float64}:
 0.5  0.9  0.25
 0.1  0.1  0.3
 0.4  0.0  0.45

In [21]:
caminho_para_probabilidade(["U1", "D1", "N"], M, ["D1", "N", "U1"], [1.0, 0.0, 0.0])

0.010000000000000002

In [71]:
function di_model(xt::Vector{T}, Q_prev::Vector{T}, t::T, Ts::T, 
                  rate_predict_t::T, num_reunioes::Int, bias::Vector{T}, 
                  a::T, b::T, gamma::T, sigma::T) where {T<:Real}

    # xt é vetor com parâmetros da matriz M
    t = t/252
    Ts = Ts/252
    M = reshape(xt, 3, 3)
    #println("M: $M")
    # Q_prev é a distribuição inicial. Ex: [0,1,0] significa 100% em N
    # Isso assume a ordem dos estados em M e mudancas_descricao é consistente.
    # Vamos gerar as descrições
    taxas = cria_vetor_mudancas(bias[1], bias[2])
    mudancas_descricao = gera_mudancas(taxas)
    # Certifique-se que a ordem em M corresponde à ordem de mudancas_descricao.
    # Ex: Se mudancas_descricao = ["D1","N","U1"], M deve ser interpretada:
    # linha = estado atual, coluna = próximo estado, na ordem [D1,N,U1].

    # Geramos todos cenários
    cenarios_distintos = calcular_cenarios_distintos(bias[1], bias[2], num_reunioes)
    #println("Cenarios Distintos: $cenarios_distintos")
    
    if length(cenarios_distintos) == 1
        return B(t, Ts, zero(T), one(T), rate_predict_t, a, b, gamma, sigma)
    end

    c_values = collect(values(cenarios_distintos))
    c_keys = collect(keys(cenarios_distintos))
    soma = zero(T)

    # Para cada conjunto de cenários (mesmo M_total), calculamos a soma das probabilidades
    for i in 1:length(c_values)
        reunioes_list = c_values[i]
        #println("Reunioes: $reunioes_list")
        prob = zero(T)
        # Para cada cenário possível
        for cenario in reunioes_list
            # cenario é algo como ["D1","N","U1"] correspondendo ao caminho D1 -> N -> U1
            p_cenario = caminho_para_probabilidade(cenario, M, mudancas_descricao, Q_prev)
            prob += p_cenario
        end
        M_total = c_keys[i]
        soma += B(t, Ts, M_total, prob, rate_predict_t, a, b, gamma, sigma)
    end

    return soma
end

di_model (generic function with 1 method)

In [73]:
bias_test = [-0.0075, -0.0025]
Q_prev_test = [0.0, 1.0, 0.0]
A_prev_test = [0.5, 0.1, 0.4, 0.9, 0.1, 0.0, 0.25, 0.30, 0.45]

9-element Vector{Float64}:
 0.5
 0.1
 0.4
 0.9
 0.1
 0.0
 0.25
 0.3
 0.45

In [75]:
di_model(A_prev_test, Q_prev_test, 1.0, 2.0, 0.103, 2, bias_test, a, b, gamma, sigma)

1.0123696707146803

In [77]:
function objective(M_t::Vector{T}, bias::Vector{T}, Q_prev::Vector{T}, t::T, Ts::T, 
                   rate_pred::T, num_reunioes::Int, alpha::T, A_prev::Vector{T}, 
                   market_price::T, a::T, b::T, gamma::T, sigma::T) where {T<:Real}
    
    prices_model = di_model(M_t, Q_prev, t, Ts, rate_pred, num_reunioes, bias, a, b, gamma, sigma)
    price_market = PU(market_price, Ts)
    
    diff_prices = price_market - prices_model
    otimizador = 0.5 * (diff_prices^2)
    penalizador = alpha * norm(M_t - A_prev)^2
    return otimizador + penalizador
end

objective (generic function with 1 method)

In [79]:
df_over = CSV.read("selic_over_jan_mar_2012.csv", DataFrame)
taxas_over = collect(skipmissing(df_over.CDI))

df_market = CSV.read("di_2012J.csv", DataFrame)
taxas_market = collect(skipmissing(df_market.close))

df_market, df_over

(62×2 DataFrame
 Row │ Column1     close   
     │ Date        Float64 
─────┼─────────────────────
   1 │ 2012-01-02  0.1039
   2 │ 2012-01-03  0.10375
   3 │ 2012-01-04  0.10385
   4 │ 2012-01-05  0.10377
   5 │ 2012-01-06  0.10345
   6 │ 2012-01-09  0.10332
   7 │ 2012-01-10  0.1032
   8 │ 2012-01-11  0.10315
   9 │ 2012-01-12  0.1031
  10 │ 2012-01-13  0.10295
  11 │ 2012-01-16  0.1029
  ⋮  │     ⋮          ⋮
  53 │ 2012-03-19  0.0953
  54 │ 2012-03-20  0.09527
  55 │ 2012-03-21  0.09528
  56 │ 2012-03-22  0.0951
  57 │ 2012-03-23  0.0948
  58 │ 2012-03-26  0.0952
  59 │ 2012-03-27  0.095
  60 │ 2012-03-28  0.0951
  61 │ 2012-03-29  0.0948
  62 │ 2012-03-30  0.0952
            41 rows omitted, 63×2 DataFrame
 Row │ Date        CDI     
     │ Date        Float64 
─────┼─────────────────────
   1 │ 2012-01-02   0.1089
   2 │ 2012-01-03   0.1088
   3 │ 2012-01-04   0.1088
   4 │ 2012-01-05   0.1085
   5 │ 2012-01-06   0.1086
   6 │ 2012-01-09   0.1082
   7 │ 2012-01-10   0.1083
   8 

In [87]:
M_test1 = [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]
M_test2 = [-2.3033499815659325e-9, 1.0000000093924066, -7.089056696700291e-9,
    -4.698077660731161e-9, 1.0000000099746342, -5.2765566326496385e-9, 2.3783922910256537e-8, 0.9999999862158437, -9.999766630295012e-9]
M_gen  = [0.73, 0.0, 0.33, 0.13, 0.87, 0.33, 0.14, 0.13, 0.34] 

A_prev = [0.33, 0.33, 0.34, 0.33, 0.34, 0.33, 0.34, 0.33, 0.33]
bias = [-0.0075, -0.0025]
Q_prev = [0.0, 1.0, 0.0]

t = 1.0
Ts = 62.0

objective(M_test1, bias, Q_prev, t, Ts,
                   taxas_over[1], 2, 0.0, A_prev, 
                   taxas_market[1], a, b, gamma, sigma), objective(M_test2, bias, Q_prev, t, Ts,
                   taxas_over[1], 2, 0.0, A_prev, 
                   taxas_market[1], a, b, gamma, sigma), objective(M_gen, bias, Q_prev, t, Ts,
                   taxas_over[1], 2, 0.0, A_prev, 
                   taxas_market[1], a, b, gamma, sigma)

(3.043760093864051e-5, 3.0437600960360224e-5, 0.12267938706002919)

In [83]:
rate_pred = taxas_over[1]
num_reunioes = 2
alpha = 0.0
market_price = taxas_market[1]
PU(market_price, 100.0), market_price

(0.9615334375528456, 0.1039)

In [91]:
# Função a ser registrada
function my_objective(x1::T, x2::T, x3::T, x4::T, x5::T, x6::T, x7::T, x8::T, x9::T) where {T<:Real}
    # Constrói a matriz M_t do tipo T
    M_t = reshape(T[x1, x2, x3, x4, x5, x6, x7, x8, x9], 3, 3)

    # Converte parâmetros externos para tipo T
    bias_T = T.(bias)
    Q_prev_T = T.(Q_prev)
    A_prev_T = T.(A_prev)
    market_price_T = T(market_price)
    a_T = T(a)
    b_T = T(b)
    gamma_T = T(gamma)
    sigma_T = T(sigma)
    rate_pred_T = T(rate_pred)
    t_T = T(t)
    Ts_T = T(Ts)
    alpha_T = T(alpha)
    
    # Chama a função objective assumindo que ela é compatível com T
    return objective(vec(M_t), bias_T, Q_prev_T, t_T, Ts_T, rate_pred_T, num_reunioes, alpha_T, A_prev_T, market_price_T, a_T, b_T, gamma_T, sigma_T)
end

model = Model(Ipopt.Optimizer)

set_optimizer_attribute(model, "print_level", 5)
set_optimizer_attribute(model, "sb", "yes")
set_optimizer_attribute(model, "max_cpu_time", 60.0)

@variable(model, M[1:3, 1:3])
@constraint(model, [i in 1:3, j in 1:3], M[i, j] >= 0)
@constraint(model, [i in 1:3, j in 1:3], M[i, j] <= 1)
@constraint(model, [i in 1:3], sum(M[i,j] for j in 1:3) == 1)

# Registra a função. Note que agora não passamos closure, mas sim a função pura definida acima.
JuMP.register(model, :my_objective, 9, my_objective, autodiff=true)

@NLobjective(model, Min, my_objective(M[1,1], M[1,2], M[1,3],
                                     M[2,1], M[2,2], M[2,3],
                                     M[3,1], M[3,2], M[3,3]))

optimize!(model)

println("Status: ", termination_status(model))
println("Solution M: ", value.(M))

LoadError: Unable to register the function :my_objective.

Common reasons for this include:

 * The function takes `f(x::Vector)` as input, instead of the splatted
   `f(x...)`.
 * The function assumes `Float64` will be passed as input, it must work for any
   generic `Real` type.
 * The function allocates temporary storage using `zeros(3)` or similar. This
   defaults to `Float64`, so use `zeros(T, 3)` instead.

As examples, instead of:
```julia
my_function(x::Vector) = sum(x.^2)
```
use:
```julia
my_function(x::T...) where {T<:Real} = sum(x[i]^2 for i in 1:length(x))
```

Instead of:
```julia
function my_function(x::Float64...)
    y = zeros(length(x))
    for i in 1:length(x)
        y[i] = x[i]^2
    end
    return sum(y)
end
```
use:
```julia
function my_function(x::T...) where {T<:Real}
    y = zeros(T, length(x))
    for i in 1:length(x)
        y[i] = x[i]^2
    end
    return sum(y)
end
```

Review the stacktrace below for more information, but it can often be hard to
understa

In [29]:
############################
# Final Objective Callable
############################

function my_objective(x::T...) where {T<:Real}
    M_array = Vector{T}(undef, length(x))
    @inbounds for i in 1:length(x)
        M_array[i] = x[i]
    end

    # Convert global data to type T
    bias_T = T.(bias)
    Q_prev_T = T.(Q_prev)
    A_prev_T = T.(A_prev)
    market_price_T = T(market_price)
    a_T = T(a)
    b_T = T(b)
    gamma_T = T(gamma)
    sigma_T = T(sigma)
    rate_pred_T = T(rate_pred)
    t_T = T(t)
    Ts_T = T(Ts)
    alpha_T = T(alpha)

    return objective(M_array, bias_T, Q_prev_T, t_T, Ts_T, rate_pred_T, num_reunioes, alpha_T, A_prev_T, market_price_T, a_T, b_T, gamma_T, sigma_T)
end

############################
# JuMP Model
############################

model = Model(Ipopt.Optimizer)

# Enable verbose output
set_optimizer_attribute(model, "print_level", 5)  # Maximum verbosity
set_optimizer_attribute(model, "sb", "yes")        # Short banner
# Optional: Redirect output to a file
# set_optimizer_attribute(model, "output_file", "ipopt_output.log")

# Optional: Set maximum CPU time (in seconds) to prevent indefinite running
set_optimizer_attribute(model, "max_cpu_time", 60.0)  # 1 minute

# Optional: Set maximum number of iterations
#set_optimizer_attribute(model, "max_iter", 1000)

@variable(model, M[1:3, 1:3])
# Add constraints to ensure each entry of M is between 0 and 1
@constraint(model, [i in 1:3, j in 1:3], M[i, j] >= 0)
@constraint(model, [i in 1:3, j in 1:3], M[i, j] <= 1)
@constraint(model, [i in 1:3], sum(M[i,j] for j in 1:3) == 1)

# Registering the function
JuMP.register(model, :my_objective, 9, (x1,x2,x3,x4,x5,x6,x7,x8,x9)->my_objective(x1,x2,x3,x4,x5,x6,x7,x8,x9), autodiff=true)

@NLobjective(model, Min, my_objective(M[1,1], M[1,2], M[1,3],
                                     M[2,1], M[2,2], M[2,3],
                                     M[3,1], M[3,2], M[3,3]))

optimize!(model)

println("Status: ", termination_status(model))
println("Solution M: ", value.(M))

LoadError: Unable to register the function :my_objective.

Common reasons for this include:

 * The function takes `f(x::Vector)` as input, instead of the splatted
   `f(x...)`.
 * The function assumes `Float64` will be passed as input, it must work for any
   generic `Real` type.
 * The function allocates temporary storage using `zeros(3)` or similar. This
   defaults to `Float64`, so use `zeros(T, 3)` instead.

As examples, instead of:
```julia
my_function(x::Vector) = sum(x.^2)
```
use:
```julia
my_function(x::T...) where {T<:Real} = sum(x[i]^2 for i in 1:length(x))
```

Instead of:
```julia
function my_function(x::Float64...)
    y = zeros(length(x))
    for i in 1:length(x)
        y[i] = x[i]^2
    end
    return sum(y)
end
```
use:
```julia
function my_function(x::T...) where {T<:Real}
    y = zeros(T, length(x))
    for i in 1:length(x)
        y[i] = x[i]^2
    end
    return sum(y)
end
```

Review the stacktrace below for more information, but it can often be hard to
understa